In [2]:
archivoVentas = 'ventasPrueba.csv'
archivoProd = 'prodPrueba.csv'

import pandas as pd
import numpy as np

from IPython.display import display
pd.options.display.max_columns = None

pd.set_option("display.max_rows",1000)

venta_df = pd.read_csv(archivoVentas)
prod_df = pd.read_csv(archivoProd, engine='python')

prod_df = prod_df.dropna()
venta_df = venta_df.dropna()


venta_df['ttmto_homol']=venta_df['Nombre_Tratamiento']

#diccionario de homologacion para VENTA y homologacion (si no esta incluida una homol, queda Nombre_Tratamiento)
di = {"RASPAJE Y PULIDO CORONAL(DETRARTAJE SUPRAGINGIVAL) X CUADRANTE": "RASPAJE_CUADRANTE", 
      "RETENEDOR FIJO 3X3 EN ALAMBRE": "RETENEDOR FIJO","RETENEDOR ESSIX O ESTETICOS": "RETENEDOR ESSIX", 
      "RETIRO BRACKETS ARCADA INFERIOR": "RETIRO BRACKETS ARCADA","RETIRO BRACKETS ARCADA SUPERIOR": "RETIRO BRACKETS ARCADA"}
venta_df['ttmto_homol'].replace(di, inplace=True)

#homologacion de PRODUCCION

prod_df['temp'] = prod_df['Tratamiento'].str.contains('MENSUALIDAD ORTODONCIA METALICA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'MENSUALIDAD ORTODONCIA METALICA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('RETIRO BRACKETS ARCADA')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETIRO BRACKETS ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('PROFILAXIS')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'PROFILAXIS'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('DETARTRAJE')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'DETARTRAJE POR ARCADA'

prod_df['temp'] = prod_df['Tratamiento'].str.contains('ESSIX')
prod_df.loc[prod_df['temp'] == True,'ttmto_homol'] = 'RETENEDOR ESSIX'


#Recorte de Fechas
venta_df['fechaApunte'] = venta_df['fechaApunte'].str[0:10]
prod_df['Fecha Cita'] = prod_df['Fecha Cita'].str[0:10]

# CAMBIAR FORMATO DE COLUMNA 'FECHA CITA' A DATE (SIN HORA) DE PROD
temp_series = pd.Series() 

temp_series = venta_df['fechaApunte']
venta_df['fechaApunte'] = pd.to_datetime(temp_series)

# concat nombre y apellido en VENTAS
venta_df['Paciente'] = venta_df['nombre'] + ' ' + venta_df['apellidos']


# Resumen de CANTIDADES en Dataframe de VENTA
ventaResumen = venta_df.groupby(['Nombre_Clinica','Paciente','dni','ttmto_homol'], as_index = False).sum()
ventaResumen['fecha_prog'] = '' #esta fecha se llena en clinica y se va corriendo auto a medida que se evoluciona, en el cruce
ventaResumen['Semana_prog']=None
ventaResumen['Semana_prod']=0 
ventaResumen['Retraso']=0
ventaResumen['Cant_prod']=0
ventaResumen['Cant_pend']=ventaResumen['Cantidad']-ventaResumen['Cant_prod']
#ventaResumen['Semanas_transcurridas']=0

#ventaResumen = ventaResumen[['Nombre_Clinica','Paciente','dni','ttmto_homol','Cantidad','Cant_prod','Cant_pend','Semana_prog']]
#ventaResumen.to_csv('progNueva.csv', index = False)
print("Archivo generado!")
venta_df


Archivo generado!


,idApunte,IdClinica,Nombre_Clinica,idContacto,nombre,apellidos,dni,Doctor,fechaApunte,Mes,Id_Tratamiento,Nombre_Tratamiento,Pieza,Cantidad,Precio,Precio_sin_descuento,Descuento_Porcentaje,Vr_descontado,Precio_con_descuento,TotalPagos,ttmto_homol,Paciente
0,37436984,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-01-02,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,6,70000,1260000,17.9,225540,1034460,50000,MENSUALIDAD ORTODONCIA METALICA,EVELYN DAZA
1,37771381,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,MARIA APONTE MUNAR,2018-08-02,February,16711,MENSUALIDAD ORTODONCIA METALICA,11,3,70000,1260000,17.9,225540,1034460,16000,MENSUALIDAD ORTODONCIA METALICA,EVELYN DAZA
2,38147237,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-02-15,February,16722,RETENEDOR FIJO 3X3 EN ALAMBRE,11,1,150000,150000,17.9,26850,123150,12650,RETENEDOR FIJO,EVELYN DAZA
3,38147242,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-02-15,February,16721,RETENEDOR ESSIX O ESTETICOS,11,1,150000,150000,17.9,26850,123150,123150,RETENEDOR ESSIX,EVELYN DAZA
4,38147247,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-02-15,February,16723,RETIRO BRACKETS ARCADA INFERIOR,31,1,100000,100000,17.9,17900,82100,82100,RETIRO BRACKETS ARCADA,EVELYN DAZA
5,38147252,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-02-15,February,16724,RETIRO BRACKETS ARCADA SUPERIOR,11,1,100000,100000,17.9,17900,82100,82100,RETIRO BRACKETS ARCADA,EVELYN DAZA
6,38944406,2000,Autopista 104A Bogota,826386,EVELYN,DAZA,111,LEIDIS MARIAN MEJIA CUENTAS,2018-02-27,February,16722,RETENEDOR FIJO 3X3 EN ALAMBRE,11,1,150000,150000,17.9,26850,123150,110500,RETENEDOR FIJO,EVELYN DAZA
7,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,2018-12-03,March,16722,PROFILAXIS,11,1,400000,40000,0.0,0,40000,40000,PROFILAXIS,PEDRO GOMEZ
8,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,2018-12-03,March,16722,PROFILAXIS,11,1,400000,40000,0.0,0,40000,40000,PROFILAXIS,PEDRO GOMEZ
9,38944409,2000,Autopista 104A Bogota,B22,PEDRO,GOMEZ,222,LEIDIS MARIAN MEJIA CUENTAS,2018-12-03,March,16722,DETARTRAJE POR ARCADA,11,2,400000,40000,0.0,0,40000,40000,DETARTRAJE POR ARCADA,PEDRO GOMEZ
